# Car Reviews

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
# import yfinance as yf
import seaborn as sns

import requests
from bs4 import BeautifulSoup
from time import time, sleep
import random
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#declare header
headers = {"Accept-Language": "en-US,en;q=0.5"}

# Define the URL of the Reuters news page
url = "https://forums.edmunds.com/discussions/tagged/x/buying/p"
start_page = 1
end_page = 100

In [4]:
#data needed
title_list = []
views_list = []
comments_list = []
links_list = []

#page range
pages = np.arange(start_page,end_page+1)

#BS4 extraction
for page in pages:
    page = requests.get(url+str(page))
    soup = BeautifulSoup(page.text, 'html.parser')
    threads = soup.findAll('div', attrs = {'class': 'ItemContent Discussion'})
    for thread in threads:
        title = thread.find('div', class_ = "Title").text.lstrip('\n').rstrip(' ')
        title_list.append(title)
       
        views = thread.find("span", class_ = 'MItem MCount ViewCount').text.lstrip('\n').rstrip(' ')
        views_list.append(views)
       
        comments = thread.find("span", class_ = 'MItem MCount CommentCount').text.lstrip('\n').rstrip(' ')
        comments_list.append(comments)

        link = thread.find('div', class_ = "Title").find('a', href=True).get('href')
        links_list.append(link)

In [5]:
# create dataframe
df_heads = pd.DataFrame({'title': title_list,
                         'views': views_list,
                         'comments': comments_list,
                         'links': links_list})

# year, make, model
df_heads['year'] = df_heads['title'].str.split(' ').str[0]
df_heads['make'] = df_heads['title'].str.split(' ').str[1]
df_heads['model'] = df_heads['title'].str.split(' ').str[2]

# comments count and views count
df_heads['commentscount'] = df_heads['comments'].str.split(' ').str[0].str.replace('K', '00').str.replace('M', '00000').str.replace('.', '').astype(int)
df_heads['viewscount'] = df_heads['views'].str.split(' ').str[0].str.replace('K', '00').str.replace('M', '00000').str.replace('.', '').astype(int)

df_heads

,title,views,comments,links,year,make,model,commentscount,viewscount
0,2020 Honda Accord Lease Deals and Prices,30.3K views,1.4K comments,https://forums.edmunds.com/discussion/58876/ho...,2020,Honda,Accord,1400,30300
1,2022 BMW X7 Lease Deals and Prices,7.6K views,244 comments,https://forums.edmunds.com/discussion/66863/bm...,2022,BMW,X7,244,7600
2,2022 RAM 1500 Lease Deals and Prices,18.6K views,1.4K comments,https://forums.edmunds.com/discussion/67158/ra...,2022,RAM,1500,1400,18600
3,2022 Jeep Grand Cherokee Lease Deals and Prices,27K views,1.7K comments,https://forums.edmunds.com/discussion/67279/je...,2022,Jeep,Grand,1700,2700
4,Mercedes Benz EQS Lease Deals and Prices,10.5K views,636 comments,https://forums.edmunds.com/discussion/67606/me...,Mercedes,Benz,EQS,636,10500
...,...,...,...,...,...,...,...,...,...
3385,What will it take for consumers to buy America...,1.2K views,1.6K comments,https://forums.edmunds.com/discussion/4712/for...,What,will,it,1600,1200
3386,Make Me a Better (Online) Car Salesman!,493 views,658 comments,https://forums.edmunds.com/discussion/4849/gen...,Make,Me,a,658,493
3387,Best Fictional Character Car Spokesperson,321 views,71 comments,https://forums.edmunds.com/discussion/4880/sub...,Best,Fictional,Character,71,321
3388,Toyota Avalon - Which Trim Line to Choose?,131 views,10 comments,https://forums.edmunds.com/discussion/15882/to...,Toyota,Avalon,-,10,131


In [6]:
df_heads.to_csv('01_df_heads.csv', index=False)

In [7]:
df_heads = pd.read_csv('01_df_heads.csv', index_col=0)
df_heads.head()

,views,comments,links,year,make,model,commentscount,viewscount
title,,,,,,,,
2020 Honda Accord Lease Deals and Prices,30.3K views,1.4K comments,https://forums.edmunds.com/discussion/58876/ho...,2020,Honda,Accord,1400,30300
2022 BMW X7 Lease Deals and Prices,7.6K views,244 comments,https://forums.edmunds.com/discussion/66863/bm...,2022,BMW,X7,244,7600
2022 RAM 1500 Lease Deals and Prices,18.6K views,1.4K comments,https://forums.edmunds.com/discussion/67158/ra...,2022,RAM,1500,1400,18600
2022 Jeep Grand Cherokee Lease Deals and Prices,27K views,1.7K comments,https://forums.edmunds.com/discussion/67279/je...,2022,Jeep,Grand,1700,2700
Mercedes Benz EQS Lease Deals and Prices,10.5K views,636 comments,https://forums.edmunds.com/discussion/67606/me...,Mercedes,Benz,EQS,636,10500


### Reviews - Comments

In [8]:
def get_data(start_index, end_index):
  df_tot = pd.DataFrame()

  for link in range(start_index, end_index):

      ## NAMES & PAGES
      #-------------------------------------------------------
      url_p = df_heads['links'][link]
      page = requests.get(url_p)
      soup = BeautifulSoup(page.text, 'html.parser')

      # Names
      groups = soup.findAll('span', attrs = {'itemprop': 'itemListElement'})
      groupsname = []
      for i,g in enumerate(groups):
        groupsname.append(g.find('span', attrs = {'itemprop': 'name'}).text)
      #print(groupsname)

      # Number of pages
      threads = soup.findAll('div', attrs = {'role': 'navigation'})
      nav_list = []
      for thread in threads:
          navig = thread.findAll('a')#.get('href')
          for nav in navig:
            nav_list.append(nav.get('href'))

      if nav_list[0]!='/discussions':
        pvec = np.zeros(len(nav_list)-3)
        for i in range(1,len(nav_list)-2):
          pvec[i-1] =  nav_list[i].replace(nav_list[0],'').replace('/p','')
        maxp = int(pvec.max())
      else:
        maxp = 1


      ## REVIEWS - COMMENTS
      #-------------------------------------------------------
      link_list = []
      group_list = []
      subgroup_list = []
      user_list = []
      role_list = []
      postcount_list = []
      datecreated_list = []
      hourcreated_list = []
      message_list = []
      reply_list = []
      nreview_list = []
      nv = 0

      threads = soup.findAll('div', attrs = {'class': 'Comment'})

      for thread in threads:
          link_list.append(link+1)
          group_list.append(groupsname[0])
          subgroup_list.append(('' if len(groupsname)<2 else groupsname[1]))
          nreview_list.append(nv+1)

          user = thread.find('span', attrs = {'class': 'Author'}).find('a').get('title')
          user_list.append(user)

          role = thread.find('span', attrs = {'class': 'MItem RoleTitle'})
          role = None if role==None else role.text
          role_list.append(role)

          postcount = int(thread.find('span', attrs = {'class': 'MItem PostCount'}).find('b').text.replace('<b>','').replace(',',''))
          postcount_list.append(postcount)

          datecreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[:10]
          datecreated_list.append(datecreated)

          hourcreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[11:19]
          hourcreated_list.append(hourcreated)

          message = thread.find('div', attrs = {'class': 'Item-Body'}).find('div', attrs = {'class': 'Message userContent'}).text#.find('p', class_='blockquote-line') #Message userContent
          message_list.append(message.replace('\n',''))

          reply = 'No' if thread.find('div', attrs = {'class': 'Item-Body'}).find('p', class_='blockquote-line') == None else 'Yes'
          reply_list.append(reply)
          nv += 1

      if maxp>1:
          for npage in range(2,maxp+1):
              page = requests.get(url_p+'/p'+str(npage))
              soup = BeautifulSoup(page.text, 'html.parser')

              threads = soup.findAll('div', attrs = {'class': 'Comment'})

              for thread in threads:
                  link_list.append(link+1)
                  group_list.append(groupsname[0])
                  subgroup_list.append(('' if len(groupsname)<2 else groupsname[1]))
                  nreview_list.append(nv+1)

                  user = thread.find('span', attrs = {'class': 'Author'}).find('a').get('title')
                  user_list.append(user)

                  role = thread.find('span', attrs = {'class': 'MItem RoleTitle'})
                  role = None if role==None else role.text
                  role_list.append(role)

                  postcount = int(thread.find('span', attrs = {'class': 'MItem PostCount'}).find('b').text.replace('<b>','').replace(',',''))
                  postcount_list.append(postcount)

                  datecreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[:10]
                  datecreated_list.append(datecreated)

                  hourcreated = thread.find('span', attrs = {'class': 'MItem DateCreated'}).find('time').get('datetime')[11:19]
                  hourcreated_list.append(hourcreated)

                  message = thread.find('div', attrs = {'class': 'Item-Body'}).find('div', attrs = {'class': 'Message userContent'}).text#.find('p', class_='blockquote-line') #Message userContent
                  message_list.append(message.replace('\n',''))

                  reply = 'No' if thread.find('div', attrs = {'class': 'Item-Body'}).find('p', class_='blockquote-line') == None else 'Yes'
                  reply_list.append(reply)
                  nv += 1

      # DataFrame
      df_aux = pd.DataFrame({'page': link_list,
                            'group': group_list,
                            'subgroup': subgroup_list,
                            'nreview': nreview_list,
                            'user': user_list,
                            'role': role_list,
                            'postcount': postcount_list,
                            'datecreated': datecreated_list,
                            'hourcreated':hourcreated_list,
                            'message':message_list,
                            'reply': reply_list})

      # df_tot = df_tot.append(df_aux)
      # append is deprecated, use concat instead
      df_tot = pd.concat([df_tot, df_aux], ignore_index=True)
      if (link+1)%50==0:
          print('NLink: {:>5}'.format(link+1))

  return df_tot


In [9]:
df_tot = get_data(0,1)
df_tot.to_csv('02_df_reviews_5.csv', index=False)


# We will use multithreading to speed up the process
# We want to call get_data() function for each rng of 100 links
# We will use 5 threads

import threading

def get_data_thread(start_index, end_index):
    df_tot = get_data(start_index, end_index)
    df_tot.to_csv('02_df_reviews_'+str(start_index)+'_'+str(end_index)+'.csv', index=False)

# # We will use 5 threads
# threads = []
# for i in range(0, 3300, 100):
#     t = threading.Thread(target=get_data_thread, args=(i, i+100))
#     threads.append(t)
#     t.start()

# for t in threads:
#     t.join()

get_data_thread(200, 300)



Bad pipe message: %s [b'\xf0\xd9\xfa\x04"(\xddv\xd9\x86\x89\x88\x03s\x047\x17\xf3 \xdft/0]v6K\xcdk@X,\xfe\xe0wn\xeeH\xb8\x9c=@\xe2\x9a\xfa\xdd\n\xc1\xcd\xf8\x1f\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00']
Bad pipe message: %s [b"k\x96=\x07\xea\xf3'\xdd\xa0\tjLF\xb1\xf6\xf4\xfe\xb1\x00\x00\x86\xc00\xc0,\xc0(\xc0$\xc0\x14\xc0\n\x00\xa5\x00\xa3\x00\xa1\x00\x9f\x00k\x00j\x00i\x00h\x009\x008\x007\x006\xc02\xc0.\xc0*\xc0&\xc0\x0f\xc0\x05\x00\x9d\x00=\x005\xc0/\xc0+\xc0'\xc0#\xc0\x13\xc0\t\x00\xa4\x00\xa2\x00\xa0\x00\x9e\x00g\x00@\x00?\x00>\x003\x002\x001\x000\xc01\xc0-\xc0)\xc0%\xc0", b'\x04\x00\x9c\x00<\x00/\x00\x9a\x00\x99\x00\x98']
Bad p

NLink:   250
NLink:   300


In [10]:
df_tot

,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply
0,1,Honda,Honda Accord,1,chait6756,Member,3,2019-11-02,07:16:09,"Hello,Can I request for the numbers like resid...",No
1,1,Honda,Honda Accord,2,Jwalker714,Member,4,2019-11-02,15:09:05,"Hi there! May I please get the MF, residual an...",No
2,1,Honda,Honda Accord,3,kyfdx,Moderator,217095,2019-11-02,15:28:12,"chait6756 said:Hello,Can I request for the num...",Yes
3,1,Honda,Honda Accord,4,Crownedjules,Member,34,2019-11-06,01:21:10,"Looking for RV, MF and incentives on 2020 Acco...",No
4,1,Honda,Honda Accord,5,Michaell,Moderator,219548,2019-11-06,07:33:53,"Crownedjules said:Looking for RV, MF and incen...",Yes
...,...,...,...,...,...,...,...,...,...,...,...
1374,1,Honda,Honda Accord,1375,kyfdx,Moderator,217095,2021-05-18,19:34:04,YaseemTheDream said:Can I get the MF/Residual/...,Yes
1375,1,Honda,Honda Accord,1376,Cam25,Member,3,2021-05-19,22:39:00,Hi there!I'm interested in a 2021 Honda Accord...,No
1376,1,Honda,Honda Accord,1377,Michaell,Moderator,219548,2021-05-20,00:38:49,Cam25 said:Hi there!I'm interested in a 2021 H...,Yes
1377,1,Honda,Honda Accord,1378,wasp,Member,2,2023-04-14,02:12:43,"Hello, Can you provide RV & MF for the 2020 Ho...",No


## Separate Message from Replies

In [1]:
import numpy as np
import pandas as pd

In [9]:
# df_reviews = pd.read_csv('02_df_reviews_1.csv')#.append(pd.read_csv('02_df_reviews_2.csv'))#.append(pd.read_csv('02_df_reviews_3.csv'))
# df_reviews['datecreated'] = pd.to_datetime(df_reviews['datecreated'])
# df_reviews['replaced_message'] = df_reviews['message']
# df_reviews.head()

for i in range(0, 3300, 100):
    start = i
    end = i + 100
    file_name = "./02_df_reviews_{}_{}.csv".format(start, end)
    try:
        df = pd.read_csv(file_name)
    except:
        continue

    df['datecreated'] = pd.to_datetime(df['datecreated'])
    df['replaced_message'] = df['message']
    # df_reviews['replaced_message'] = df_reviews.apply(lambda row: replace_text(row), axis=1)
    df['replaced_message'] = df.apply(lambda row: replace_text(row, df), axis=1)

    df.to_csv(file_name.replace('.csv', '_replaced.csv'), index=False)
    display(df.head())

,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,1.0,Honda,Honda Accord,1.0,chait6756,Member,3.0,2019-11-02,07:16:09,"Hello,Can I request for the numbers like resid...",No,"Hello,Can I request for the numbers like resid..."
1,1.0,Honda,Honda Accord,2.0,Jwalker714,Member,4.0,2019-11-02,15:09:05,"Hi there! May I please get the MF, residual an...",No,"Hi there! May I please get the MF, residual an..."
2,1.0,Honda,Honda Accord,3.0,kyfdx,Moderator,217095.0,2019-11-02,15:28:12,"chait6756 said:Hello,Can I request for the num...",Yes,"chait6756 said:Hello,Can I request for the num..."
3,1.0,Honda,Honda Accord,4.0,Crownedjules,Member,34.0,2019-11-06,01:21:10,"Looking for RV, MF and incentives on 2020 Acco...",No,"Looking for RV, MF and incentives on 2020 Acco..."
4,1.0,Honda,Honda Accord,5.0,Michaell,Moderator,219540.0,2019-11-06,07:33:53,"Crownedjules said:Looking for RV, MF and incen...",Yes,.00150 and 57%$500 dealer incentive


,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,101.0,Hyundai,Hyundai Kona,1.0,Michaell,Moderator,219540.0,2022-09-02,22:41:10,NJustice36 said:looking for 2023 kona lease nu...,Yes,NJustice36 said:looking for 2023 kona lease nu...
1,101.0,Hyundai,Hyundai Kona,2.0,laya27,Member,3.0,2022-09-10,00:41:58,"Can I please get the MF, RV and lease incentiv...",No,"Can I please get the MF, RV and lease incentiv..."
2,101.0,Hyundai,Hyundai Kona,3.0,Michaell,Moderator,219540.0,2022-09-10,01:15:56,"laya27 said:Can I please get the MF, RV and le...",Yes,N-Line - .00243 MF and 62% RVSEL - .00187 and...
3,101.0,Hyundai,Hyundai Kona,4.0,robmos,Member,3.0,2022-09-10,17:37:05,Could I please get SEL '23 Kona Lease numbers ...,No,Could I please get SEL '23 Kona Lease numbers ...
4,101.0,Hyundai,Hyundai Kona,5.0,kyfdx,Moderator,217095.0,2022-09-10,19:27:16,robmos said:Could I please get SEL '23 Kona Le...,Yes,FWD or AWD?What term and mileage allowance?


/tmp/ipykernel_1088/3820408685.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,201.0,General,NaN,1.0,willow25,Member,2.0,2001-09-25,01:58:22,"I'm buying a ""new"" car by the end of the year ...",No,"I'm buying a ""new"" car by the end of the year ..."
1,201.0,General,NaN,2.0,dindak,Member,6632.0,2001-10-04,13:09:27,Buy a 2001/ 2002 Oldsmobile Intrigue. It's one...,No,Buy a 2001/ 2002 Oldsmobile Intrigue. It's one...
2,201.0,General,NaN,3.0,peteri1,Member,21.0,2001-10-04,14:45:33,Am looking at 1 of 4 choices to replace my 99G...,No,Am looking at 1 of 4 choices to replace my 99G...
3,201.0,General,NaN,4.0,mrdetailer,Member,1118.0,2001-10-04,15:15:29,Generally a 3-4 year old car range is a good o...,No,Generally a 3-4 year old car range is a good o...
4,201.0,General,NaN,5.0,dindak,Member,6632.0,2001-10-04,15:38:06,Maxima and Intrigue are both excellent choices...,No,Maxima and Intrigue are both excellent choices...


,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,301.0,Mercedes-Benz,Mercedes-Benz C-Class,1.0,kyfdx,Moderator,217095.0,2020-11-12,13:01:16,"carluvs said:What's the MF, Residual, and leas...",Yes,"carluvs said:What's the MF, Residual, and leas..."
1,301.0,Mercedes-Benz,Mercedes-Benz C-Class,2.0,xqlailai,Member,6.0,2020-11-16,14:59:32,Hello! Can I please get advise on the below de...,No,Hello! Can I please get advise on the below de...
2,301.0,Mercedes-Benz,Mercedes-Benz C-Class,3.0,kyfdx,Moderator,217095.0,2020-11-16,15:04:59,xqlailai said:Hello! Can I please get advise o...,Yes,"Seems like a reasonable deal for that vehicle,..."
3,301.0,Mercedes-Benz,Mercedes-Benz C-Class,4.0,xqlailai,Member,6.0,2020-11-16,15:13:02,kyfdx said: xqlailai said:Hello! Can I please ...,Yes,"Yes, 7,500/year is enough for me. Do you thin..."
4,301.0,Mercedes-Benz,Mercedes-Benz C-Class,5.0,kyfdx,Moderator,217095.0,2020-11-16,15:24:54,xqlailai said: kyfdx said: xqlailai said:Hello...,Yes,"Yes, .00070 is correct"


...

,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,2801,Honda,Honda Civic,1,bri70,Member,147,2000-02-17,19:10:23,"After considering both cars, I went with the H...",No,"After considering both cars, I went with the H..."
1,2801,Honda,Honda Civic,2,umchand2,Member,7,2000-02-18,08:48:24,before i bought my elntra i was also deciding ...,No,before i bought my elntra i was also deciding ...
2,2801,Honda,Honda Civic,3,taurus2,Member,63,2000-03-05,11:30:04,Is the elantra reliable?,No,Is the elantra reliable?
3,2801,Honda,Honda Civic,4,scape2,Member,4123,2000-03-08,03:33:27,I quoted some months back some very impressive...,No,I quoted some months back some very impressive...
4,2801,Honda,Honda Civic,5,isellhondas,Member,20341,2000-03-09,08:42:57,Hyundai went with the 10 year warranty as a de...,No,Hyundai went with the 10 year warranty as a de...


,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,2901,General,NaN,1,prithis,Member,13,2008-05-20,18:44:20,This is a Bill of Sale format that I used whil...,No,This is a Bill of Sale format that I used whil...
1,2901,General,NaN,2,calleaghn,Member,4,2008-05-20,19:28:09,I live in California and just bought a car on ...,No,I live in California and just bought a car on ...
2,2901,General,NaN,3,tidester,Member,10059,2008-05-21,00:48:13,I would be very reluctant to provide a strange...,No,I would be very reluctant to provide a strange...
3,2901,General,NaN,4,Kirstie_H,Administrator,11127,2008-05-21,00:57:38,"well, if it is a dealer and the cash transacti...",No,"well, if it is a dealer and the cash transacti..."
4,2901,General,NaN,5,calleaghn,Member,4,2008-05-21,03:08:42,"since asking the question, i've gathered some ...",No,"since asking the question, i've gathered some ..."


,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,3001,Toyota,Toyota 4Runner,1,gwapo,Member,20,2001-06-16,23:28:21,It's so easy to decide when you've got much mo...,No,It's so easy to decide when you've got much mo...
1,3001,Toyota,Toyota 4Runner,2,johndoe6,Member,12,2001-06-17,02:53:16,If you like the trucklike ride in an SUV then ...,No,If you like the trucklike ride in an SUV then ...
2,3001,Toyota,Toyota 4Runner,3,verywell,Member,1,2001-06-18,21:26:44,"Re 4-Runner as Highlander, 4-runner will be a ...",No,"Re 4-Runner as Highlander, 4-runner will be a ..."
3,3001,Toyota,Toyota 4Runner,4,mikenhgolf,Member,6,2001-06-25,00:05:01,"off road stuff, then pick the 4Runner. If you ...",No,"off road stuff, then pick the 4Runner. If you ..."
4,3001,Toyota,Toyota 4Runner,5,pillars,Member,1,2001-06-27,02:02:17,"I drove a Highlander about a month ago, fully ...",No,"I drove a Highlander about a month ago, fully ..."


,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,3101,Lexus,Lexus IS 250,1,bdw,Member,2,2007-07-29,19:35:36,I am in the same bort but don't have any new i...,No,I am in the same bort but don't have any new i...
1,3101,Lexus,Lexus IS 250,2,kcleuk,Member,45,2007-07-30,02:25:29,"As far as I know, the new IS-F will be a lot o...",No,"As far as I know, the new IS-F will be a lot o..."
2,3101,Lexus,Lexus IS 250,3,06nighthawkv6,Member,55,2007-10-21,10:34:26,no 6MT for IS 350??; crying shame if not...whe...,No,no 6MT for IS 350??; crying shame if not...whe...
3,3101,Lexus,Lexus IS 250,4,epaiva,Member,3,2008-06-27,23:27:00,I can't find any information about the release...,No,I can't find any information about the release...
4,3102,Acura,Acura MDX,1,stever,Guest,52454,2007-07-06,05:15:50,Try the Importing Car into Canada from US disc...,No,Try the Importing Car into Canada from US disc...


,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
0,3201.0,General,NaN,1.0,driftracer,Member,2448.0,2003-10-27,19:32:55,who photograph their cars before and after ser...,No,who photograph their cars before and after ser...
1,3201.0,General,NaN,2.0,isellhondas,Member,20341.0,2003-10-27,20:00:44,I pulled up Honda and the state I'm in. It onl...,No,I pulled up Honda and the state I'm in. It onl...
2,3201.0,General,NaN,3.0,driftracer,Member,2448.0,2003-10-27,20:07:12,of what I said before - I used to manage a tir...,No,of what I said before - I used to manage a tir...
3,3201.0,General,NaN,4.0,gadgetcar,Member,3.0,2003-10-27,20:08:02,"Imagine the ""pleasure"" of the customer who is ...",No,"Imagine the ""pleasure"" of the customer who is ..."
4,3201.0,General,NaN,5.0,blh7068,Member,375.0,2003-10-27,20:45:22,"""Imagine the ""pleasure"" of the customer who is...",No,"""Imagine the ""pleasure"" of the customer who is..."


In [8]:
# Define function to replace text of previous row on each row
def replace_text(row, df):
    if row.name == 0:
        return row['message']
    elif pd.isnull(df.loc[row.name,'message']) or pd.isnull(df.loc[row.name-1,'message']):
        return ''
    elif df.loc[row.name-1, 'message'][0]=='@':
        b = (10 if len(df.loc[row.name-1, 'message'])>10 else len(df.loc[row.name-1, 'message']))
        return df.loc[row.name, 'message'].split(df.loc[row.name-1, 'message'][-b:])[-1]
    else:
        a_message = df.loc[row.name-1,'message'].lstrip(' ').rstrip(' ')
        a_replaced_message = df.loc[row.name-1, 'replaced_message'].lstrip(' ').rstrip(' ')
        a_usersaid = str(('@' + df.loc[row.name-1, 'user'] if df.loc[row.name, 'message'][0]=='@' else df.loc[row.name-1, 'user'])) + ' said:'

        return df.loc[row.name, 'message'].replace(a_message,'').replace(a_replaced_message,'').replace(a_usersaid, '')

In [ ]:
# Apply function to create new column with replaced text
df_reviews['replaced_message'] = df_reviews.apply(lambda row: replace_text(row), axis=1)

In [ ]:
df_reviews[-20:]

,page,group,subgroup,nreview,user,role,postcount,datecreated,hourcreated,message,reply,replaced_message
21459,100,Toyota,Toyota Sienna,51,08gpws2k,Member,1,2023-03-13,02:50:26,"Can some one with a Toyota lease ""calculator"" ...",No,"Can some one with a Toyota lease ""calculator"" ..."
21460,100,Toyota,Toyota Sienna,52,Michaell,Moderator,219239,2023-03-13,03:17:40,08gpws2k said:Can some one with a Toyota lease...,Yes,What is your location?What is the selling pric...
21461,100,Toyota,Toyota Sienna,53,kellerexpress,Member,6,2023-03-18,17:46:26,"Hello, can I please get the MF, incentives and...",No,"Hello, can I please get the MF, incentives and..."
21462,100,Toyota,Toyota Sienna,54,kyfdx,Moderator,216892,2023-03-18,17:48:56,"kellerexpress said:Hello, can I please get the...",Yes,36/12.00225 MF and 63% residualNo lease incent...
21463,100,Toyota,Toyota Sienna,55,kellerexpress,Member,6,2023-03-18,17:57:35,"kyfdx said: kellerexpress said:Hello, can I pl...",Yes,Thank you! So if the dealer is quoting me a h...
21464,100,Toyota,Toyota Sienna,56,kyfdx,Moderator,216892,2023-03-18,18:41:50,kellerexpress said: kyfdx said: kellerexpress ...,Yes,Possibly marking up the money factor.
21465,100,Toyota,Toyota Sienna,57,BrMn,Member,1,2023-03-19,17:32:24,"Hi,Please help with MF and RV for 2023 Sienna ...",No,"Hi,Please help with MF and RV for 2023 Sienna ..."
21466,100,Toyota,Toyota Sienna,58,kyfdx,Moderator,216892,2023-03-19,18:09:29,"BrMn said:Hi,Please help with MF and RV for 20...",Yes,.00225 MF62%/63% residual for XLE/XSEWe don't...
21467,100,Toyota,Toyota Sienna,59,kodimus1,Member,1,2023-03-22,07:54:03,"Can you please provide the Residual, MF for th...",No,"Can you please provide the Residual, MF for th..."
21468,100,Toyota,Toyota Sienna,60,kyfdx,Moderator,216892,2023-03-22,12:30:55,kodimus1 said:Can you please provide the Resid...,Yes,FWD or AWD?
